Example of how to create custom dataset in Pytorch. In this case
we have images of cats and dogs in a separate folder and a csv
file containing the name to the jpg file as well as the target
label (0 for cat, 1 for dog).

Programmed by Aladdin Persson <aladdin.persson at hotmail dot com>
*    2020-04-03 Initial coding
*    2022-12-19 Updated with better comments, improved code using PIL, and checked code still functions as intended.

In [1]:
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from PIL import Image
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches

/home/thien/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = Image.open(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 3e-4
batch_size = 32
num_epochs = 10

In [5]:
# Load Data
dataset = CatsAndDogsDataset(
    csv_file="7_Custom_Dataset_Images/cats_dogs.csv",
    root_dir="7_Custom_Dataset_Images/cats_dogs_resized",
    transform=transforms.ToTensor(),
)

In [15]:
dataset.__getitem__(2)

(tensor([[[0.2549, 0.2549, 0.2000,  ..., 0.5216, 0.5882, 0.5216],
          [0.2275, 0.2392, 0.2000,  ..., 0.5804, 0.6980, 0.5647],
          [0.3098, 0.2863, 0.2549,  ..., 0.5490, 0.5569, 0.5765],
          ...,
          [0.5176, 0.3725, 0.2392,  ..., 0.1529, 0.1804, 0.0980],
          [0.5882, 0.4471, 0.2510,  ..., 0.1843, 0.1490, 0.5490],
          [0.5686, 0.4510, 0.2706,  ..., 0.1216, 0.5020, 0.7686]],
 
         [[0.2078, 0.2078, 0.1529,  ..., 0.4980, 0.5529, 0.4863],
          [0.1765, 0.1882, 0.1490,  ..., 0.5529, 0.6627, 0.5294],
          [0.2588, 0.2353, 0.2000,  ..., 0.5137, 0.5216, 0.5333],
          ...,
          [0.4784, 0.3333, 0.2000,  ..., 0.1098, 0.1412, 0.0627],
          [0.5608, 0.4196, 0.2118,  ..., 0.1373, 0.1059, 0.5059],
          [0.5412, 0.4235, 0.2314,  ..., 0.0745, 0.4588, 0.7255]],
 
         [[0.2157, 0.2157, 0.1608,  ..., 0.4353, 0.4863, 0.4196],
          [0.2039, 0.2157, 0.1765,  ..., 0.4824, 0.5961, 0.4627],
          [0.3255, 0.3020, 0.2588,  ...,

In [14]:
dataset.__len__()

10

In [26]:
# Dataset is actually a lot larger ~25k images, just took out 10 pictures
# to upload to Github. It's enough to understand the structure and scale
# if you got more images.
train_set, test_set = torch.utils.data.random_split(dataset, [5, 5])  # needs attribute .__len__() of the dataset
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Model
model = torchvision.models.googlenet(weights="DEFAULT")

# freeze all layers, change final linear layer with num_classes
for param in model.parameters():
    param.requires_grad = False

# final layer is not frozen
model.fc = nn.Linear(in_features=1024, out_features=num_classes)
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)



In [27]:
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):  # needs .__getitem__() attribute of dataset
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")



Cost at epoch 0 is 0.7233263850212097
Cost at epoch 1 is 0.6048968434333801
Cost at epoch 2 is 0.5148124694824219
Cost at epoch 3 is 0.5543022751808167
Cost at epoch 4 is 0.5021189451217651
Cost at epoch 5 is 0.45927420258522034
Cost at epoch 6 is 0.4333237111568451
Cost at epoch 7 is 0.38993144035339355
Cost at epoch 8 is 0.35386139154434204
Cost at epoch 9 is 0.3682589828968048


In [28]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

In [12]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Checking accuracy on Training Set
Got 4 / 5 with accuracy 80.00
Checking accuracy on Test Set
Got 4 / 5 with accuracy 80.00
